In [2]:
#import model
#!pip install lime
#!pip install tensorflow transformers
#tf.keras.backend.clear_session()

In [4]:
#LOAD IN MODEL (if needed)

##One thing is check that the classification layer is of size 5 and not size 6.
import pandas as pd
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf

checkpoint = 'roberta-base'
roberta_tokenizer = RobertaTokenizer.from_pretrained(checkpoint)

file_path = "../models/roberta_model_final.keras"

# Load the model
roberta_classification_model = tf.keras.models.load_model(file_path, custom_objects={'TFRobertaModel': TFRobertaModel})

# Display the model summary
roberta_classification_model.summary()

2024-07-25 17:03:36.781456: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-25 17:03:36.782125: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids_layer (InputLayer)   [(None, None)]       0           []                               
                                                                                                  
 attention_mask_layer (InputLay  [(None, None)]      0           []                               
 er)                                                                                              
                                                                                                  
 tf_roberta_model (TFRobertaMod  TFBaseModelOutputWi  124645632  ['input_ids_layer[0][0]',        
 el)                            thPoolingAndCrossAt               'attention_mask_layer[0][0]']   
                                tentions(last_hidde         

In [8]:
import pandas as pd
test_data = pd.read_csv('../efcamdat_test_data.csv')

test_data['cefr_numeric'] = test_data['cefr_numeric'].astype('category')

test_texts = test_data["text"]
test_labels = test_data["cefr_numeric"].apply(lambda x: int(x)-1)


In [9]:
#test_data["cefr_numeric"].value_counts()
num_classes = test_labels.nunique()
num_classes

5

In [10]:
test_instance_index = 1
test_instance = test_texts.iloc[test_instance_index]
true_label = test_labels.iloc[test_instance_index]

In [11]:
print(f"Text: {test_instance_index} True Label: {true_label}")

Text: 1 True Label: 1


In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

test_instance_no_stopwords = remove_stopwords(test_instance)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielskahill/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
print(test_instance_no_stopwords)

Hello! name Julia I'll talk rotine. afternoon, Sean plays baskteboll every day, Granny laundry every Tuesday walk dog. feed dog 8pm every day. Granny gardening morning. 5pm, Sean watches movies Saturdays fees dog every day. 6pm Granny sets table Sean plays computer games.


In [15]:
import numpy as np
MAX_SEQUENCE_LENGTH = 128

def predict_prob(text):
    encodings = roberta_tokenizer(text, truncation=True, padding='max_length', max_length=MAX_SEQUENCE_LENGTH, return_tensors='tf')
    inputs = {
        'input_ids_layer': encodings['input_ids'],
        #'token_type_ids': encodings['token_type_ids'],
        'attention_mask_layer': encodings['attention_mask']
    }
    predictions = roberta_classification_model(inputs, training=False)
    return tf.argmax(predictions, axis=-1).numpy()

# Get the predicted label for the test instance
predicted_label = predict_prob(test_instance_no_stopwords)[0]

print(f"Predicted label: {predicted_label}")
print(f"True label: {true_label}")

Predicted label: 1
True label: 1


In [ ]:
import lime
from lime.lime_text import LimeTextExplainer

# Initialize the LIME text explainer
explainer = LimeTextExplainer(class_names=list(test_labels.cat.categories))

# Explain the model's prediction on the test instance
exp = explainer.explain_instance(test_instance_no_stopwords, predict_prob, num_features=5)

# Display the explanation with true and predicted labels
print(f"True label: {test_labels.cat.categories[true_label]}")
print(f"Predicted label: {test_labels.cat.categories[predicted_label]}")
exp.show_in_notebook(text=True)

In [ ]:
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns

weights = OrderedDict(exp.as_list())
lime_weights = pd.DataFrame({"words": list(weights.keys()), "weights": list(weights.values())})

sns.barplot(x = "words", y = "weights", data = lime_weights)
plt.xticks(rotation = 45)
plt.title("Sample {} features weights given by LIME".format(test_instance_index))
plt.show()